In [18]:
import time

import numpy as np

import lsst.verify

In [19]:
metric_file = "sysver.yaml"
metric_name = "sysver.CameraBodySurfaceTemp"

In [20]:
# This is how a single metric file is read
sysver_metrics = lsst.verify.MetricSet.load_single_package(metric_file)
#sysver_metrics

In [21]:
def measure_temperature(temperature_offset=None, percentage=0.1):
    # Parameters for a Gaussian temparature distribution
    mu = 10 # Celsius
    sigma = 0.1 # Celsius
    num_measurements = 1000

    temp = np.random.normal(mu, sigma, num_measurements)
    
    if temperature_offset is not None:
        frac = int(percentage * num_measurements)
        value = np.random.randint(0, num_measurements, frac)
        indexes = np.unique(value)
        temp[indexes] += temperature_offset
    
    avg_temp = np.mean(temp)
    return avg_temp, temp

In [29]:
def run_normal_job(iterations=10, interval=60):
    for i in range(iterations):
        average_temperature, temperature_data = measure_temperature()
        avg_temp_meas = lsst.verify.Measurement("sysver.CameraBodySurfaceTemp", average_temperature)
        avg_temp_meas.notes['estimator'] = 'numpy.mean'
        print(average_temperature)
        if i < iterations - 1:
            time.sleep(interval)

In [36]:
def run_drift_job(iterations=10, interval=60):
    temperature_ramp = 0.75
    fraction_ramp = 0.1
    for i in range(iterations):
        temperature_offset = temperature_ramp * i
        fraction_change = fraction_ramp * i
        average_temperature, temperature_data = measure_temperature(temperature_offset, fraction_change)
        avg_temp_meas = lsst.verify.Measurement("sysver.CameraBodySurfaceTemp", average_temperature)
        avg_temp_meas.notes['estimator'] = 'numpy.mean'
        print(average_temperature)
        if i < iterations - 1:
            time.sleep(interval)

In [37]:
#run_normal_job()

In [39]:
#run_drift_job(interval=1)